# BaT Data Farming

In [329]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import joblib
import os
import sys
import helpers

Get List of Every Result and Save:

In [517]:
# Get List of Every Result and Save
make = input('Enter Car Make: ')
model = input('Enter Car Model: ')
open("./data/HTML/"+ make + model +".html", 'w').close()
ids, urls = helpers.getlistings(make, model)
data = ''
listings = []

# asdfasdfasdf
def get_data(url):
    r = requests.get(url, allow_redirects=False)
    soup = BeautifulSoup(r.content, "html.parser")
    return str(soup)

# Iterate through BaT specific Id's and show more pages for each listing
urlend = '&results=items'
pgend = 15
delimiter = "%newline%"
for i in range(0,len(ids)):
    urlstart = 'https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?bat_keyword_pages=' + ids[i] + '&sort=td&page='
    tempurl = helpers.geturls(urlstart, urlend, pgend, urls)
    listings = tempurl
    listings = listings[len(ids):len(listings)]
    souplist = joblib.Parallel(n_jobs=-2)(joblib.delayed(get_data)(url) for url in listings)

    temp = delimiter.join(souplist)
    # data = temp + delimiter + data
    data += delimiter + temp 

# Write data to file 
with open("./data/HTML/"+ make + model +".html", "a") as file:
    file.write(data)

display('done')


'done'

Pull Information Off Each Listing Stored in URLS List:

In [521]:
carlist = []
Car = []
### READ DATA ###

with open("./data/HTML/"+ make + model +".html", "r") as file:
    test_data = file.read()

### Write Data to DF ###
out = test_data.split(delimiter)

for i in range(0,len(listings)):
    soup = BeautifulSoup(out[i], 'html.parser')
    title = str(soup.find('title'))
    if (
        "Parts" not in title 
        and "Tool" not in title 
        and "Memorabilia" not in title 
        and "Luggage" not in title 
        and "Tool Kit" not in title 
        and "Removable" not in title 
        and "Gearbox" not in title 
        and title != 'None' 
        and 'listing' in listings[i]
    ):
        loc = str(soup.findAll('a'))
        transmission = str(soup.findAll('li'))
        contents = str(soup.findAll('p'))
        essentials = soup.find('div', class_='essentials')

        #Get Car Description
        desc = helpers.getdesc(listings[i])

        #Get Year
        year = desc[0:4]
        
        #Check if Sold and get highest bid
        sold, price = helpers.getsaleprice(title, loc)

        #Get Transmission
        trans = 'Manual'
        if 'Automatic' in transmission:
            trans = 'Automatic'

        #Get Location
        town, state = helpers.getlocation(loc)
        
        #Check for Project
        if 'project' in title or 'Project' in title:
            project = 1
        else:
            project = 0
        
        #Get Lot Number
        try:
            index = title.index('Lot #')
            lotnumb = title[index+5:index+12].partition(')')[0]
        except:
            lotnumb = "N/A"

        #Check for Replica
        if "replica" in title or "Replica" in title:
            replica = 1
        else:
            replica = 0

        #Get Month
        month = helpers.getmonth(title)
        
        #Get Year
        if '(' in title:
            index = title.index('(')
            yearsold = title[index-5:index-1]
        else:
            index = title.index('20')
            yearsold = title[index:index+5]
        
        #Get Engine and Number Matching
        numbmatch = 0
        if 'Numbers-Matching' in transmission:
            numbmatch = 1
        engine = helpers.getengine(transmission)
        
        #Get Various Indicators for rust, refurbishment, restoration, scratch, paint bubbles, metal repair, hardtop, or overdrive options
        rust, refurbished, restored, scratch, paintbub, metalrepair, hardtop, overdrive, turbo, super = helpers.getindicators(contents)
            
        #Get Mileage
        miles, milestmu = helpers.getmileage(transmission)

        #Get Descriptions of options and chassis #
        chassis, specialdesc, mileagedesc, enginedesc, transdesc, paintdesc, interiordesc, carbdesc, wheeldesc, brakedesc, suspdesc, engine = helpers.getenginedesc(essentials, engine)

        #Check Trans with Description
        if 'Manual' in transdesc:
            trans = 'Manual'
        elif 'Automatic' in transdesc:
            trans = 'Automatic'

        #Check Mileage with Description (TODO: remove try catch)
        try:
            if "K" in str(miles):
                index = mileagedesc.index("ilometers")
                tempmile = mileagedesc[0:index].partition(" ")[0]
                if "k" in tempmile or "K" in tempmile:
                    milestmu = re.sub(r"[^0-9]", "", tempmile)
                    milestmu = int(milestmu) * 1000
            elif "K" in str(milestmu):
                index = mileagedesc.index("ilometers")
                tempmile = mileagedesc[0:index].partition(" ")[0]
                if "k" in tempmile or "K" in tempmile:
                    miles = re.sub(r"[^0-9]", "", tempmile)
                    miles = int(miles) * 1000
        except:
            print('failed')
            
        #Check for bespoke bodies
        if 'Bertone' in title:
            bertone = 1
        else:
            bertone = 0
        if 'Zagato' in title:
            zagato = 1
        else:
            zagato = 0

        Car = [lotnumb, desc, trans, year, miles, milestmu, engine, turbo, super, numbmatch, hardtop, overdrive, scratch, paintbub, metalrepair, replica, rust, refurbished, restored, project, sold, month, yearsold, price, town, state, zagato, bertone, chassis, specialdesc, mileagedesc, enginedesc, transdesc, paintdesc, interiordesc, carbdesc, wheeldesc, brakedesc, suspdesc, listings[i], make, model]
        
        carlist.append(Car)
    
    else:
        print('Discarded')

# Export to DataFrame
df = pd.DataFrame(carlist, columns=['Lot Number', 'Description', 'Transmission', 'Model Year', 'Mileage', 'Mileage (if TMU)','Engine', 'Turbocharged', 'Supercharged', 'Numbers Matching', 'Hardtop', 'Overdrive', 'Scratch', 'Paint Bubble', 'Metal Repair', 'Replica', 'Rust','Refurbished','Restored', 'Project', 'Sold', 'Month', 'Year', 'Price', 'City', 'State', 'Zagato', 'Bertone', 'Chassis', 'Special Description', 'Mileage Description', 'Engine Description', 'Transmission Description','Paint Description', 'Interior Description','Carburetor Description', 'Wheels Description', 'Brakes Description', 'Suspension Description', 'URL', 'Make', 'Model'])

display(df)



,Lot Number,Description,Transmission,Model Year,Mileage,Mileage (if TMU),Engine,Turbocharged,Supercharged,Numbers Matching,...,Transmission Description,Paint Description,Interior Description,Carburetor Description,Wheels Description,Brakes Description,Suspension Description,URL,Make,Model
0,"75,351",1967 ford cortina,Manual,1967,TMU,10000,1.6,0,0,0,...,Spare Engine Blocks & Transmission,N/A,N/A,N/A,N/A,N/A,N/A,https://bringatrailer.com/listing/1967-ford-co...,Ford,Cortina
1,"63,290",1968 ford cortina,Manual,1968,TMU,58000,3.0,0,0,0,...,5-Speed Manual Transmission,N/A,N/A,N/A,"Black-Finished 15"" Alloy Wheels",N/A,N/A,https://bringatrailer.com/listing/1968-ford-co...,Ford,Cortina
2,"55,760",1966 ford cortina gt,Manual,1966,TMU,76000,1.6,0,0,0,...,Four-Speed Manual Transmission,Red Paint w/Silver Accents,Black Vinyl Upholstery,N/A,"13"" Wheels",N/A,Lowered Suspension,https://bringatrailer.com/listing/1966-ford-co...,Ford,Cortina
3,"54,075",1963 ford cortina,Manual,1963,TMU - K,45000,1.2,0,0,0,...,Four-Speed Manual Transmission,N/A,Red Upholstery,N/A,N/A,N/A,N/A,https://bringatrailer.com/listing/1963-ford-co...,Ford,Cortina
4,"51,989",1967 ford cortina,Manual,1967,TMU,38000,1.6,0,0,0,...,Five-Speed Manual Transmission,N/A,Red Leather Upholstery,Dual Weber Carburetors,"Black 16"" Honda S2000 Wheels",Wilwood Disc Brakes,Aftermarket Suspension Components,https://bringatrailer.com/listing/1967-ford-co...,Ford,Cortina
5,"45,407",1967 ford cortina estate,Manual,1967,TMU,62000,1.5,0,0,0,...,Four-Speed Manual Transmission,N/A,N/A,N/A,Additional Wheels Included,N/A,N/A,https://bringatrailer.com/listing/1967-ford-co...,Ford,Cortina
6,"41,383",1967 ford cortina,Manual,1967,TMU,84000,2.0,0,0,0,...,Ford T9 5-Speed Manual Transmission,Repainted Red w/ Yellow Stripe,N/A,N/A,"14"" Minilite-Style Wheels",N/A,N/A,https://bringatrailer.com/listing/1967-ford-co...,Ford,Cortina
7,"40,803",1963 ford cortina gt,Manual,1963,TMU - K,,1.6,0,0,0,...,5-Speed Manual Gearbox,N/A,Black Upholstery w/ Red Piping,Weber Carburetors,"13"" Maxilite Wheels",Front Disc Brakes,N/A,https://bringatrailer.com/listing/1963-ford-co...,Ford,Cortina
8,"40,386",1965 ford cortina gt,Manual,1965,TMU,42000,2.0,0,0,0,...,Honda Six-Speed Manual Transmission,N/A,Black Interior,N/A,N/A,Four-Wheel Disc Brakes,Custom Four-Link Rear Suspension,https://bringatrailer.com/listing/1965-ford-co...,Ford,Cortina
9,"38,096",1968 ford cortina,Manual,1968,TMU,12000,1.6,0,0,0,...,4-Speed Manual Transmission,N/A,Black Interior,Dual Weber Carburetors,Slotted Aluminum Wheels,N/A,N/A,https://bringatrailer.com/listing/1968-ford-co...,Ford,Cortina


Save Data as Excel File:

In [519]:
path = 'data/' + make + '/'

try:
    os.mkdir(path)
except:
    print('Path already exists')

df.to_excel(path + make + '_' + model + '.xlsx')

print('Saved!')

Path already exists
Saved!
